In [4]:

select FORMAT(SUM(VALOR_TRANSACAO), 'C') AS TOTAL from ORIGEM_DESTINO

(1 row affected)

Total execution time: 00:00:00.018

TOTAL
"R$ 11.380.750,00"


In [14]:
SELECT t.NOME_TITULAR AS NOME, FORMAT(t.VALOR_RENDA, 'C') AS RENDA, t.CPF_CNPJ_TITULAR AS 'CPF/CNPJ',  t.TELEFONE_PESSOA AS FONE, 

FORMAT(ISNULL(
(SELECT SUM(VALOR_LANCAMENTO) AS CREDITOS
    FROM EXTRATO e 
    WHERE e.NUMERO_BANCO = t.NUMERO_BANCO 
    AND e.NUMERO_AGENCIA = t.NUMERO_AGENCIA 
    AND e.NUMERO_CONTA LIKE t.NUMERO_CONTA 
    AND e.TIPO_LANCAMENTO BETWEEN 201 AND 219 ) 
-    
(SELECT SUM(VALOR_LANCAMENTO)  AS DEBITOS
    FROM EXTRATO e 
    WHERE e.NUMERO_BANCO = t.NUMERO_BANCO 
    AND e.NUMERO_AGENCIA = t.NUMERO_AGENCIA 
    AND e.NUMERO_CONTA LIKE t.NUMERO_CONTA 
    AND e.TIPO_LANCAMENTO BETWEEN 101 AND 122 ), '0'),
    'C') AS MOVIMENTOU

FROM TITULARES t ORDER BY NOME

(6 rows affected)

Total execution time: 00:00:00.130

NOME,RENDA,CPF/CNPJ,FONE,MOVIMENTOU
EMPRESA DO INVESTIGADO 200,"R$ 300.002,00",59717553000617,3138989898,"-R$ 3.775.450,00"
FULANO DA SILVA (PESSOA NO MESMO BANCO),"R$ 0,00",55555555121,,"R$ 0,00"
INVESTIGADO 200,"R$ 400.000,00",20050506870,6140004545,"R$ 554.800,00"
INVESTIGADO 765,"R$ 400.000,00",76527514700,6140004545,"-R$ 3.775.450,00"
JOAO ALMEIDA,"R$ 0,00",35463200009,,"R$ 0,00"
MARLENE MATOS,"R$ 500.000,00",12345678909,6248989898,"R$ 0,00"


In [16]:
SELECT t.NOME_TITULAR AS NOME, FORMAT(t.VALOR_RENDA, 'C') AS RENDA, t.CPF_CNPJ_TITULAR AS 'CPF/CNPJ',  t.TELEFONE_PESSOA AS FONE, 

FORMAT(ISNULL(
(SELECT TOP(1)VALOR_SALDO 
    FROM EXTRATO e 
    WHERE e.NUMERO_BANCO = t.NUMERO_BANCO 
    AND e.NUMERO_AGENCIA = t.NUMERO_AGENCIA 
    AND e.NUMERO_CONTA LIKE t.NUMERO_CONTA 
    AND e.TIPO_LANCAMENTO BETWEEN 201 AND 219
    ORDER BY e.CODIGO_CHAVE_EXTRATO DESC) 
, '0'),
    'C') AS MOVIMENTOU

FROM TITULARES t ORDER BY NOME

(6 rows affected)

Total execution time: 00:00:00.125

NOME,RENDA,CPF/CNPJ,FONE,MOVIMENTOU
EMPRESA DO INVESTIGADO 200,"R$ 300.002,00",59717553000617,3138989898,"R$ 4.383.850,00"
FULANO DA SILVA (PESSOA NO MESMO BANCO),"R$ 0,00",55555555121,,"R$ 0,00"
INVESTIGADO 200,"R$ 400.000,00",20050506870,6140004545,"R$ 5.914.800,00"
INVESTIGADO 765,"R$ 400.000,00",76527514700,6140004545,"R$ 4.383.850,00"
JOAO ALMEIDA,"R$ 0,00",35463200009,,"R$ 0,00"
MARLENE MATOS,"R$ 500.000,00",12345678909,6248989898,"R$ 0,00"


In [18]:
SELECT NOME_AGENCIA AS NOME, NUMERO_BANCO+'/'+NUMERO_AGENCIA AS 'BANCO/AGÊNCIA', ENDERECO_CIDADE AS CIDADE, ENDERECO_CEP AS CEP, ENDERECO_UF AS UF, ENDERECO_PAIS AS 'PAÍS', TELEFONE_AGENCIA AS FONE, DATA_ABERTURA_AGENCIA AS ABERTURA, ISNULL(DATA_FECHAMENTO_AGENCIA, '-') AS FECHAMENTO FROM AGENCIAS ORDER BY NOME_AGENCIA

: Msg 8114, Level 16, State 5, Line 1
Erro ao converter tipo de dados varchar em numeric.

Total execution time: 00:00:00.253

NOME,BANCO/AGÊNCIA,CIDADE,CEP,UF,PAÍS,FONE,ABERTURA,FECHAMENTO


In [10]:
 SELECT DISTINCT
ROW_NUMBER() OVER (ORDER BY e.DATA_LANCAMENTO, e.CODIGO_CHAVE_EXTRATO ASC) AS 'SEQUENCE',
e.CODIGO_CHAVE_EXTRATO AS 'ID_LANCAMENTO',
FORMAT(e.VALOR_LANCAMENTO, 'C') AS 'VALOR_LANÇAMENTO',
ISNULL(l.CODIGO, 0) as 'TIPO_LANCAMENTO_N',
l.DESCRICAO AS 'TIPO_DE_LANÇAMENTO',
e.NUMERO_DOCUMENTO AS 'N_DOCUMENTO',
e.NUMERO_BANCO as 'BANCO_EXTRATO',
a.NOME_AGENCIA AS 'AGÊNCIA_EXTRATO',
e.NUMERO_CONTA AS 'CONTA_EXTRATO',
FORMAT(e.VALOR_SALDO, 'C') AS 'SALDO',
t.NOME_TITULAR AS 'NOME EXTRATO' ,
CONVERT(VARCHAR,e.DATA_LANCAMENTO, 103) AS 'DATA_DE_LANÇAMENTO',
FORMAT(o.VALOR_TRANSACAO, 'C') AS 'VALOR_TRANSAÇÃO',
o.NOME_PESSOA_OD AS 'NOME_OD',
o.NUMERO_BANCO_OD AS 'BANCO_OD',
o.NUMERO_AGENCIA_OD AS 'AGÊNCIA_OD',
od.NOME_AGENCIA AS 'NOME_AGENCIA_OD',
o.NUMERO_CONTA_OD AS 'CONTA_OD',
o.NOME_ENDOSSANTE_CHEQUE AS 'NOME_ENDOSSSANTE_CHEQUE',
o.DOC_ENDOSSANTE_CHEQUE AS 'DOC_ENDOSSANTE_CHEQUE',
o.OBSERVACAO,
e.DATA_LANCAMENTO, 
e.CODIGO_CHAVE_EXTRATO
FROM EXTRATO e 
INNER JOIN ORIGEM_DESTINO o ON e.CODIGO_CHAVE_EXTRATO = o.CODIGO_CHAVE_EXTRATO 
INNER JOIN AGENCIAS a ON a.NUMERO_BANCO = e.NUMERO_BANCO AND a.NUMERO_AGENCIA = e.NUMERO_AGENCIA 
INNER JOIN TITULARES t ON t.NUMERO_BANCO = e.NUMERO_BANCO AND t.NUMERO_AGENCIA = e.NUMERO_AGENCIA AND t.NUMERO_CONTA = e.NUMERO_CONTA
LEFT JOIN AGENCIAS od ON od.NUMERO_BANCO = o.NUMERO_BANCO_OD AND od.NUMERO_AGENCIA = o.NUMERO_AGENCIA_OD
LEFT JOIN TIPO_LANCAMENTO l ON e.TIPO_LANCAMENTO = l.CODIGO
order by e.DATA_LANCAMENTO, e.CODIGO_CHAVE_EXTRATO

(43 rows affected)

Total execution time: 00:00:00.070

SEQUENCE,ID_LANCAMENTO,VALOR_LANÇAMENTO,TIPO_LANCAMENTO_N,TIPO_DE_LANÇAMENTO,N_DOCUMENTO,BANCO_EXTRATO,AGÊNCIA_EXTRATO,CONTA_EXTRATO,SALDO,NOME EXTRATO,DATA_DE_LANÇAMENTO,VALOR_TRANSAÇÃO,NOME_OD,BANCO_OD,AGÊNCIA_OD,NOME_AGENCIA_OD,CONTA_OD,NOME_ENDOSSSANTE_CHEQUE,DOC_ENDOSSANTE_CHEQUE,OBSERVACAO,DATA_LANCAMENTO,CODIGO_CHAVE_EXTRATO
1,1,"R$ 100.000,00",219,Pagamentos Salariais,123,888,AGENCIA 101,111111,"R$ 5.100.000,00",INVESTIGADO 200,01/01/2006,"R$ 100.000,00",EMPRESA,1,101,NULL,4455,,,DINHEIRO,2006-01-01 00:00:00.000,1
2,2,"R$ 100.000,00",201,Depósitos,124,888,AGENCIA 101,111111,"R$ 5.200.000,00",INVESTIGADO 200,01/01/2006,"R$ 50.000,00",ENVOLVIDO 4,0,0,NULL,,,,,2006-01-01 00:00:00.000,2
3,2,"R$ 100.000,00",201,Depósitos,124,888,AGENCIA 101,111111,"R$ 5.200.000,00",INVESTIGADO 200,01/01/2006,"R$ 50.000,00",ENVOLVIDO 4,0,0,NULL,,,,,2006-01-01 00:00:00.000,2
4,3,"R$ 45.250,00",105,Tarifas,101,888,AGENCIA 101,111111,"R$ 5.154.750,00",INVESTIGADO 200,01/01/2006,"R$ 45.250,00",NOME DO CEDENTE,0,0,NULL,,,,,2006-01-01 00:00:00.000,3
5,4,"R$ 400.000,00",117,Transferência entre Contas,0122,888,AGENCIA 101,111111,"R$ 5.114.750,00",INVESTIGADO 200,01/01/2006,"R$ 400.000,00",MARLENE MATOS,888,101,AGENCIA 101,222221,,,,2006-01-01 00:00:00.000,4
6,11,"R$ 200.000,00",201,Depósitos,2123,888,AGENCIA 103,333331,"R$ 5.914.800,00",INVESTIGADO 765,01/01/2006,"R$ 150.000,00",FULANO DA SILVA,888,102,AGENCIA 102,456555,,,,2006-01-01 00:00:00.000,11
7,11,"R$ 200.000,00",201,Depósitos,2123,888,AGENCIA 103,333331,"R$ 5.914.800,00",EMPRESA DO INVESTIGADO 200,01/01/2006,"R$ 150.000,00",FULANO DA SILVA,888,102,AGENCIA 102,456555,,,,2006-01-01 00:00:00.000,11
8,11,"R$ 200.000,00",201,Depósitos,2123,888,AGENCIA 103,333331,"R$ 5.914.800,00",INVESTIGADO 765,01/01/2006,"R$ 50.000,00",CICLANO DE SOUZA,1,2315,NULL,9799,,,,2006-01-01 00:00:00.000,11
9,11,"R$ 200.000,00",201,Depósitos,2123,888,AGENCIA 103,333331,"R$ 5.914.800,00",EMPRESA DO INVESTIGADO 200,01/01/2006,"R$ 50.000,00",CICLANO DE SOUZA,1,2315,NULL,9799,,,,2006-01-01 00:00:00.000,11
10,12,"R$ 150.000,00",201,Depósitos,212301,888,AGENCIA 103,333331,"R$ 6.064.800,00",INVESTIGADO 765,03/01/2006,"R$ 150.000,00",FULANO DA SILVA,888,102,AGENCIA 102,456555,,,,2006-01-03 00:00:00.000,12


In [32]:
SELECT e.NUMERO_AGENCIA AS 'AGENCIA_A', 
e.NUMERO_CONTA AS 'CONTA_A',
o.NUMERO_BANCO_OD AS 'BANCO_B', 
o.NUMERO_CONTA_OD 'CONTA_B', 
e.TIPO_LANCAMENTO,
o.VALOR_TRANSACAO,
t.DESCRICAO,
o.CODIGO_CHAVE_OD,
CONVERT(VARCHAR, e.DATA_LANCAMENTO, 103) AS 'DATA_LANCAMENTO'
FROM ORIGEM_DESTINO o
INNER JOIN EXTRATO e ON e.CODIGO_CHAVE_EXTRATO = o.CODIGO_CHAVE_EXTRATO
INNER JOIN TIPO_LANCAMENTO t ON e.TIPO_LANCAMENTO = t.CODIGO
ORDER BY DATA_LANCAMENTO



(26 rows affected)

Total execution time: 00:00:00.126

AGENCIA_A,CONTA_A,BANCO_B,CONTA_B,TIPO_LANCAMENTO,VALOR_TRANSACAO,DESCRICAO,CODIGO_CHAVE_OD,DATA_LANCAMENTO
101,111111,1,4455,219,100000,Pagamentos Salariais,1001,01/01/2006
101,111111,0,,201,50000,Depósitos,1002,01/01/2006
101,111111,0,,201,50000,Depósitos,1003,01/01/2006
101,111111,0,,105,45250,Tarifas,1004,01/01/2006
101,111111,888,222221,117,400000,Transferência entre Contas,1005,01/01/2006
103,333331,888,456555,201,150000,Depósitos,1014,01/01/2006
103,333331,1,9799,201,50000,Depósitos,1015,01/01/2006
101,111111,104,77777,209,100050,"Transferência Interbancária (DOC, TED)",1006,01/02/2006
101,111111,0,,114,100000,Saque Eletrônico,1009,01/02/2006
103,333331,888,07303,201,450000,Depósitos,1023,01/02/2006


In [14]:
SELECT CONVERT(VARCHAR,NUMERO_AGENCIA_OD)+'/'+CONVERT(VARCHAR,NUMERO_CONTA_OD)  AS NODE FROM ORIGEM_DESTINO UNION SELECT CONVERT(VARCHAR,NUMERO_AGENCIA)+'/'+CONVERT(VARCHAR,NUMERO_CONTA)  FROM EXTRATO

(13 rows affected)

Total execution time: 00:00:00.005

NODE
0/
101/111111
101/222221
101/4455
102/07303
102/456555
103/333331
2122/9890980
2311/001004445
2315/9799


In [18]:
SELECT CONVERT(CHAR, max(DATA_LANCAMENTO), 103), CONVERT(CHAR, min(DATA_LANCAMENTO), 103) FROM extrato

(1 row affected)

Total execution time: 00:00:00.004

(No column name),(No column name)
01/05/2006,01/01/2006


In [20]:
SELECT DB_NAME() AS [Current Database];  

(1 row affected)

Total execution time: 00:00:00.297

Current Database
SINCRIFI


In [7]:
SELECT t.NOME_TITULAR AS 'NOME TITULAR', t.NUMERO_BANCO AS 'BANCO', t.NUMERO_AGENCIA AS 'AGÊNCIA', t.NUMERO_CONTA AS 'CONTA', a.NOME_AGENCIA AS 'NOME AGÊNCIA', c.DATA_ABERTURA_CONTA AS 'DATA DE ABERTURA', c.DATA_ENCERRAMENTO_CONTA AS 'DATA DE ENCERRAMENTO'  FROM TITULARES t
INNER JOIN AGENCIAS a ON a.NUMERO_BANCO = t.NUMERO_BANCO AND a.NUMERO_AGENCIA = t.NUMERO_AGENCIA
INNER JOIN CONTAS c ON c.NUMERO_BANCO = a.NUMERO_BANCO AND c.NUMERO_AGENCIA = t.NUMERO_AGENCIA 

(10 rows affected)

Total execution time: 00:00:00.123

NOME TITULAR,BANCO,AGÊNCIA,CONTA,NOME AGÊNCIA,DATA DE ABERTURA,DATA DE ENCERRAMENTO
FULANO DA SILVA (PESSOA NO MESMO BANCO),888,102,456555,AGENCIA 102,NULL,NULL
FULANO DA SILVA (PESSOA NO MESMO BANCO),888,102,456555,AGENCIA 102,NULL,NULL
JOAO ALMEIDA,888,102,07303,AGENCIA 102,NULL,NULL
JOAO ALMEIDA,888,102,07303,AGENCIA 102,NULL,NULL
INVESTIGADO 765,888,103,333331,AGENCIA 103,2001-01-01 00:00:00.000,NULL
INVESTIGADO 200,888,101,111111,AGENCIA 101,2001-01-01 00:00:00.000,2006-12-31 00:00:00.000
INVESTIGADO 200,888,101,111111,AGENCIA 101,2001-01-01 00:00:00.000,NULL
MARLENE MATOS,888,101,222221,AGENCIA 101,2001-01-01 00:00:00.000,2006-12-31 00:00:00.000
MARLENE MATOS,888,101,222221,AGENCIA 101,2001-01-01 00:00:00.000,NULL
EMPRESA DO INVESTIGADO 200,888,103,333331,AGENCIA 103,2001-01-01 00:00:00.000,NULL


In [22]:
SELECT * FROM ORIGEM_DESTINO

(27 rows affected)

Total execution time: 00:00:00.106

CODIGO_CHAVE_OD,CODIGO_CHAVE_EXTRATO,VALOR_TRANSACAO,NUMERO_DOCUMENTO_TRANSACAO,NUMERO_BANCO_OD,NUMERO_AGENCIA_OD,NUMERO_CONTA_OD,TIPO_CONTA_OD,TIPO_PESSOA_OD,CPF_CNPJ_OD,NOME_PESSOA_OD,NOME_DOC_IDENTIFICACAO_OD,NUMERO_DOC_IDENTIFICACAO_OD,CODIGO_DE_BARRAS,NOME_ENDOSSANTE_CHEQUE,DOC_ENDOSSANTE_CHEQUE,SITUACAO_IDENTIFICACAO,OBSERVACAO
1001,1,100000,123,1,101,4455,1,2,1210313000137,EMPRESA,DE CARRO,,,,,0,DINHEIRO
1002,2,50000,1241,0,0,,0,1,0,ENVOLVIDO 4,RG,442456 SSP/PE,,,,0,
1003,2,50000,1242,0,0,,0,1,0,ENVOLVIDO 4,RG,442456 SSP/PE,,,,0,
1004,3,45250,101,0,0,,0,0,1666666000146,NOME DO CEDENTE,,,84610000001058100100509051499836863214040508,,,0,
1005,4,400000,0122,888,101,222221,1,1,12345678909,MARLENE MATOS,,,,,,0,
1006,5,100050,4654,104,2344,77777,1,2,11112222333359,CALDOS E COMPANHIA SA,,,,,,0,
1009,6,100000,46546,0,0,,0,0,0,,,,,,,0,
1010,7,100000,041651,0,0,,0,0,0,RECEBEDOR DO CHEQUE FINAL,,,,,,0,
1011,8,100000,254,0,0,,0,0,78943213204,PESSOA BENEFICIARIA DO CHEQUE FINAL,,,,NOME DO ENDOSSANTE,RG 12334 SSP/DF,0,SAQUE EM ESPECIE
1012,9,900000,126,888,103,333331,1,2,11112222111100,PJ MATERIAIS DE CONSTRUÇÃO,,,,,,0,


In [2]:
SELECT CONVERT(CHAR, MIN(DATA_LANCAMENTO), 103) MINNIMA, CONVERT(CHAR, MAX(DATA_LANCAMENTO), 103) AS MAXIMA FROM EXTRATO

(1 row affected)

Total execution time: 00:00:00.021

MINNIMA,MAXIMA
01/01/2006,01/05/2006


In [3]:
SELECT 
                                o.CODIGO_CHAVE_EXTRATO, 
                                CONVERT(INT, e.TIPO_LANCAMENTO) AS TIPO_LANCAMENTO, 
                                CONVERT(CHAR, e.DATA_LANCAMENTO, 103) AS DATA_LANCAMENTO,
                                CONVERT(CHAR, o.VALOR_TRANSACAO) AS VALOR_TRANSACAO,
                                CONVERT(CHAR, e.NUMERO_BANCO) AS NUMERO_BANCO, 
                                CONVERT(CHAR, e.NUMERO_AGENCIA) AS NUMERO_AGENCIA, 
                                CONVERT(CHAR, e.NUMERO_CONTA) AS NUMERO_CONTA,
                                t.NOME_TITULAR,
                                CONVERT(CHAR, o.NUMERO_BANCO_OD) AS NUMERO_BANCO_OD, 
                                CONVERT(CHAR, o.NUMERO_AGENCIA_OD) AS NUMERO_AGENCIA_OD, 
                                CONVERT(CHAR, o.NUMERO_CONTA_OD) AS NUMERO_CONTA_OD, 
                                CONVERT(CHAR, o.NOME_PESSOA_OD) AS NOME_PESSOA_OD 
                                FROM EXTRATO e
                                INNER JOIN ORIGEM_DESTINO o ON e.CODIGO_CHAVE_EXTRATO = o.CODIGO_CHAVE_EXTRATO
                                INNER JOIN TITULARES t ON e.NUMERO_BANCO = t.NUMERO_BANCO AND e.NUMERO_AGENCIA = t.NUMERO_AGENCIA  
                                AND e.NUMERO_CONTA = t.NUMERO_CONTA
                                where e.DATA_LANCAMENTO BETWEEN CONVERT(DATE, '2006-01-01') AND CONVERT(DATE, '2006-05-01')
                                ORDER BY e.DATA_LANCAMENTO

(43 rows affected)

Total execution time: 00:00:00.102

CODIGO_CHAVE_EXTRATO,TIPO_LANCAMENTO,DATA_LANCAMENTO,VALOR_TRANSACAO,NUMERO_BANCO,NUMERO_AGENCIA,NUMERO_CONTA,NOME_TITULAR,NUMERO_BANCO_OD,NUMERO_AGENCIA_OD,NUMERO_CONTA_OD,NOME_PESSOA_OD
11,201,01/01/2006,150000,888,103,333331,INVESTIGADO 765,888,102,456555,FULANO DA SILVA
11,201,01/01/2006,150000,888,103,333331,EMPRESA DO INVESTIGADO 200,888,102,456555,FULANO DA SILVA
11,201,01/01/2006,50000,888,103,333331,INVESTIGADO 765,1,2315,9799,CICLANO DE SOUZA
11,201,01/01/2006,50000,888,103,333331,EMPRESA DO INVESTIGADO 200,1,2315,9799,CICLANO DE SOUZA
1,219,01/01/2006,100000,888,101,111111,INVESTIGADO 200,1,101,4455,EMPRESA
2,201,01/01/2006,50000,888,101,111111,INVESTIGADO 200,0,0,,ENVOLVIDO 4
2,201,01/01/2006,50000,888,101,111111,INVESTIGADO 200,0,0,,ENVOLVIDO 4
3,105,01/01/2006,45250,888,101,111111,INVESTIGADO 200,0,0,,NOME DO CEDENTE
4,117,01/01/2006,400000,888,101,111111,INVESTIGADO 200,888,101,222221,MARLENE MATOS
12,201,03/01/2006,150000,888,103,333331,INVESTIGADO 765,888,102,456555,FULANO DA SILVA


In [4]:
SELECT * FROM 
(SELECT 
    DISTINCT
    (SELECT TOP(1) NOME_TITULAR FROM TITULARES WHERE NUMERO_BANCO = e.NUMERO_BANCO AND NUMERO_AGENCIA = e.NUMERO_AGENCIA AND NUMERO_CONTA = e.NUMERO_CONTA ) AS 'TITULAR A',
    CONCAT(e.NUMERO_BANCO,' / ',e.NUMERO_AGENCIA,' / ',e.NUMERO_CONTA) AS 'BANCO / AGÊNCIA / CONTA',
    DESCRICAO AS 'TIPO DE LANÇAMENTO',
    e.DESCRICAO_LANCAMENTO AS 'DESCRIÇÃO DO LANÇAMENTO',
    FORMAT(o.VALOR_TRANSACAO, 'C') AS VALOR,
    CONVERT(CHAR, e.DATA_LANCAMENTO, 103) AS 'DATA',
    o.NOME_PESSOA_OD AS 'TITULAR B',
    CONCAT(o.NUMERO_BANCO_OD,' / ',o.NUMERO_AGENCIA_OD,' / ',o.NUMERO_CONTA_OD) AS 'BANCO / AGÊNCIA / CONTA',
    LOCAL_TRANSACAO AS 'LOCAL DA TRANSAÇÃO',
    o.NOME_ENDOSSANTE_CHEQUE AS 'NOME ENDOSSANTE CHEQUE',
    o.DOC_ENDOSSANTE_CHEQUE AS 'DOC ENDOSSANTE CHEQUE',
    o.CODIGO_DE_BARRAS AS 'CÓDIGO DE BARRAS',
    o.OBSERVACAO AS 'OBSERVAÇÃO'
    FROM EXTRATO e
    INNER JOIN ORIGEM_DESTINO o ON e.CODIGO_CHAVE_EXTRATO = o.CODIGO_CHAVE_EXTRATO
    INNER JOIN TIPO_LANCAMENTO t ON t.CODIGO = e.TIPO_LANCAMENTO
    WHERE 
    (e.NUMERO_AGENCIA = '0' AND e.NUMERO_CONTA = '' AND o.NUMERO_AGENCIA_OD = '101' AND o.NUMERO_CONTA_OD = '111111') 
    OR 
    (e.NUMERO_AGENCIA = '101' AND e.NUMERO_CONTA = '111111' AND o.NUMERO_AGENCIA_OD = '0' AND o.NUMERO_CONTA_OD = '') 
    AND
    e.DATA_LANCAMENTO BETWEEN '01/01/2006' AND '2006-05-01'
)
ORDER BY 

(3 rows affected)

Total execution time: 00:00:00.023

TITULAR A,BANCO / AGÊNCIA / CONTA,TIPO DE LANÇAMENTO,DESCRIÇÃO DO LANÇAMENTO,VALOR,DATA,TITULAR B,BANCO / AGÊNCIA / CONTA,LOCAL DA TRANSAÇÃO,NOME ENDOSSANTE CHEQUE,DOC ENDOSSANTE CHEQUE,CÓDIGO DE BARRAS,OBSERVAÇÃO
INVESTIGADO 200,888 / 101 / 111111,Depósitos,DEPOSITO,"R$ 50.000,00",01/01/2006,ENVOLVIDO 4,0 / 0 /,AGENCIA TESTE - BRASILIA/DF,,,,
INVESTIGADO 200,888 / 101 / 111111,Depósitos,DEPOSITO,"R$ 50.000,00",01/01/2006,ENVOLVIDO 4,0 / 0 /,AGENCIA TESTE - BRASILIA/DF,,,,
INVESTIGADO 200,888 / 101 / 111111,Tarifas,PGTO LUZ,"R$ 45.250,00",01/01/2006,NOME DO CEDENTE,0 / 0 /,TELEFONE,,,84610000001058100100509051499836863214040508,


In [9]:
SELECT * FROM ORIGEM_DESTINO o
INNER JOIN EXTRATO e ON e.CODIGO_CHAVE_EXTRATO = o.CODIGO_CHAVE_EXTRATO

(27 rows affected)

Total execution time: 00:00:00.030

CODIGO_CHAVE_OD,CODIGO_CHAVE_EXTRATO,VALOR_TRANSACAO,NUMERO_DOCUMENTO_TRANSACAO,NUMERO_BANCO_OD,NUMERO_AGENCIA_OD,NUMERO_CONTA_OD,TIPO_CONTA_OD,TIPO_PESSOA_OD,CPF_CNPJ_OD,NOME_PESSOA_OD,NOME_DOC_IDENTIFICACAO_OD,NUMERO_DOC_IDENTIFICACAO_OD,CODIGO_DE_BARRAS,NOME_ENDOSSANTE_CHEQUE,DOC_ENDOSSANTE_CHEQUE,SITUACAO_IDENTIFICACAO,OBSERVACAO,CODIGO_CHAVE_EXTRATO,NUMERO_BANCO,NUMERO_AGENCIA,NUMERO_CONTA,TIPO_CONTA,DATA_LANCAMENTO,NUMERO_DOCUMENTO,DESCRICAO_LANCAMENTO,TIPO_LANCAMENTO,VALOR_LANCAMENTO,NATUREZA_LANCAMENTO,VALOR_SALDO,NATUREZA_SALDO,LOCAL_TRANSACAO
1001,1,100000,123,1,101,4455,1,2,1210313000137,EMPRESA,DE CARRO,,,,,0,DINHEIRO,1,888,101,111111,1,2006-01-01 00:00:00.000,123,PAGTO SALARIO,219,100000,C,5100000,C,AGENCIA TESTE - BRASILIA/DF
1002,2,50000,1241,0,0,,0,1,0,ENVOLVIDO 4,RG,442456 SSP/PE,,,,0,,2,888,101,111111,1,2006-01-01 00:00:00.000,124,DEPOSITO,201,100000,C,5200000,C,AGENCIA TESTE - BRASILIA/DF
1003,2,50000,1242,0,0,,0,1,0,ENVOLVIDO 4,RG,442456 SSP/PE,,,,0,,2,888,101,111111,1,2006-01-01 00:00:00.000,124,DEPOSITO,201,100000,C,5200000,C,AGENCIA TESTE - BRASILIA/DF
1004,3,45250,101,0,0,,0,0,1666666000146,NOME DO CEDENTE,,,84610000001058100100509051499836863214040508,,,0,,3,888,101,111111,1,2006-01-01 00:00:00.000,101,PGTO LUZ,105,45250,D,5154750,C,TELEFONE
1005,4,400000,0122,888,101,222221,1,1,12345678909,MARLENE MATOS,,,,,,0,,4,888,101,111111,1,2006-01-01 00:00:00.000,0122,TRANSFER,117,400000,D,5114750,C,INTERNET
1006,5,100050,4654,104,2344,77777,1,2,11112222333359,CALDOS E COMPANHIA SA,,,,,,0,,5,888,101,111111,1,2006-02-01 00:00:00.000,4654,TED OUTRO BCO,209,100050,C,5214800,C,AGENCIA DA SERRA
1009,6,100000,46546,0,0,,0,0,0,,,,,,,0,,6,888,101,111111,1,2006-02-01 00:00:00.000,46546,SAQUE TERMINAL,114,100000,D,5114800,C,PAB ALTAMIRA - BELEM/PA
1010,7,100000,041651,0,0,,0,0,0,RECEBEDOR DO CHEQUE FINAL,,,,,,0,,7,888,101,111111,1,2006-03-01 00:00:00.000,041651,CHEQUE DESCONTADO,101,100000,D,5014800,C,AGENCIA DA LAGOA - RIO DE JANEIRO
1011,8,100000,254,0,0,,0,0,78943213204,PESSOA BENEFICIARIA DO CHEQUE FINAL,,,,NOME DO ENDOSSANTE,RG 12334 SSP/DF,0,SAQUE EM ESPECIE,8,888,101,111111,1,2006-04-01 00:00:00.000,254,CHEQUE DESCONTADO,101,100000,D,4914800,C,AGENCIA DA SERRA
1012,9,900000,126,888,103,333331,1,2,11112222111100,PJ MATERIAIS DE CONSTRUÇÃO,,,,,,0,,9,888,101,111111,1,2006-05-01 00:00:00.000,126,DEPOSITO CHEQUE,201,900000,C,5814800,C,
